Team SASS - Final IPYNB File

Link to presentation and files - https://drive.google.com/drive/folders/1m8FXm0XZ-6hF2uosKARgbTXmq0EiNNty

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
!pip install contextualSpellCheck
import contextualSpellCheck

import string
import re
import spacy


import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import nltk
import nltk.corpus
nltk.download()
import re
import string
from nltk.corpus import stopwords
import gensim

from google.colab import drive
drive.mount('/drive')

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopwords
      Package stopwords is already up-to-date!

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q
Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [2]:
filePath = '/drive/My Drive/Colab Notebooks/Hack-at-Luddy/'

train_df = pd.read_csv(filePath+'train_dataset.csv', encoding = "ISO-8859-1")
test_df = pd.read_csv(filePath+'test_dataset.csv', encoding = "ISO-8859-1")

In [3]:
test_df['Target'] = np.nan

In [4]:
train_df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'key', 'ID', 'create_date', 'user',
       'userID', 'RT_TF', 'full_text', 'Sample.ID.x', 'Sample.ID.y',
       'Still.Exists.x', 'Still.Exists.y', 'In.English.x', 'In.English.y',
       'Sarcasm.x', 'Sarcasm.y', 'Additional.Comments.x',
       'Additional.Comments.y', 'User.x', 'User.y', 'Antisemitism.Rating.x',
       'Antisemitism.Rating.y', 'Disagree.With.x', 'Disagree.With.y',
       'Sentiment.Rating.x', 'Sentiment.Rating.y', 'Calling.Out.x',
       'Calling.Out.y', 'Is.About.the.Holocaust.x', 'Is.About.the.Holocaust.y',
       'IHRA.Section.x', 'IHRA.Section.y', 'sample_name',
       'Is.About.The.Holocaust.x', 'Is.About.The.Holocaust.y', 'Target'],
      dtype='object')

In [5]:
train_df['Is.About.the.Holocaust.x'] = train_df['Is.About.the.Holocaust.x'].replace(np.nan,0)
train_df['Is.About.the.Holocaust.y'] = train_df['Is.About.the.Holocaust.y'].replace(np.nan,0)

train_df['Is.About.The.Holocaust.x'] = train_df['Is.About.The.Holocaust.x'].replace(np.nan,0)
train_df['Is.About.The.Holocaust.y'] = train_df['Is.About.The.Holocaust.y'].replace(np.nan,0)


In [6]:
train_df['Is.About.the.Holocaust.x'] = train_df[["Is.About.the.Holocaust.x", "Is.About.The.Holocaust.x"]].max(axis=1)
train_df['Is.About.the.Holocaust.y'] = train_df[["Is.About.the.Holocaust.y", "Is.About.The.Holocaust.y"]].max(axis=1)


In [7]:
train_df = train_df.drop('Is.About.The.Holocaust.x', axis = 1)
train_df = train_df.drop('Is.About.The.Holocaust.y', axis = 1)
train_df = train_df.drop('Unnamed: 0.1', axis = 1)
train_df = train_df.drop('Unnamed: 0', axis = 1)

In [8]:
train_df[['Target','Sentiment.Rating.x']].corr()

,Target,Sentiment.Rating.x
Target,1.00000,-0.58991
Sentiment.Rating.x,-0.58991,1.00000


In [9]:
training_df = train_df[['full_text','Target']]
testing_df = test_df[['full_text','Target']]
print(training_df.shape)
print(testing_df.shape)

(3216, 2)
(804, 2)


In [10]:
trainDF, testDF = train_test_split(training_df, test_size = 0.2)
testDF, validDF = train_test_split(testDF, test_size = 0.2)

In [11]:
print(trainDF.shape)
print(testDF.shape)
print(validDF.shape)

(2572, 2)
(515, 2)
(129, 2)


In [12]:
total_data= pd.concat([training_df,testing_df],axis=0,ignore_index=True)

In [13]:
def remove_url(text):
    url=re.compile(r'https?://S+|www\.\S+')
    return url.sub(r'',text)
def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)

In [14]:
total_data['full_text']=total_data['full_text'].apply(lambda x:remove_url(x))
total_data['full_text']=total_data['full_text'].apply(lambda x:remove_html(x))

In [15]:
for i in range(len(total_data)):
    total_data['full_text'][i].lower()

In [16]:
stop_words=set(stopwords.words("english"))
print(stop_words)

{'mustn', 'am', 'for', 'some', 'off', 'hadn', 'will', 'hasn', 'and', "that'll", 'isn', 'himself', 'had', 'nor', 'weren', 'own', 'whom', 'o', "haven't", 'of', "needn't", "you'd", 'until', 'them', 'ma', 'mightn', 'on', 'my', 'yourself', 'by', 'those', 'below', 'her', 'a', 'no', 'ourselves', 'down', 'doesn', 'then', 'that', 'an', 'while', 'as', 'few', "isn't", "wasn't", 'very', 'too', "you're", 'more', 'yourselves', 'each', 'herself', 'over', 'with', 'further', 'theirs', 'any', 'where', 'i', 'through', 'when', 'don', 'against', 'to', 'you', 'how', 'once', 'not', 've', "hasn't", 'it', "weren't", 'out', "don't", 'both', 'm', 'll', 'now', 'our', 'just', 'didn', "she's", 'again', 'he', 'into', "hadn't", 'being', 'if', 'here', 'at', 'under', 'who', 'y', 'these', 'does', 'won', "you'll", 'be', "aren't", 'she', 'which', 'up', "doesn't", 're', 'wouldn', 's', 'him', 'shouldn', 'why', 'shan', 'been', 'what', 'haven', 'ours', 'aren', 'can', 'should', 'about', "it's", 'its', 'there', 'me', 'their', '

In [17]:
total_data['tokenized_text']=total_data['full_text'].apply(gensim.utils.simple_preprocess)

In [18]:
total_data.head()

,full_text,Target,tokenized_text
0,Israel starts the new year by demolishing a Pa...,0.0,"[israel, starts, the, new, year, by, demolishi..."
1,@realDonaldTrump Trump the White Supremacist...,0.0,"[realdonaldtrump, trump, the, white, supremaci..."
2,Please read this. International blackmail from...,1.0,"[please, read, this, international, blackmail,..."
3,Netanyahu accidentally calls Israel a 'nuclear...,0.0,"[netanyahu, accidentally, calls, israel, nucle..."
4,Jews werent getting their asses kicked on the...,0.0,"[jews, weren, getting, their, asses, kicked, o..."


In [19]:
def remove_stopwords(s):
    cleantext=[]
    for i in range(len(s)): 
        if(s[i] not in stop_words):
                cleantext.append(s[i])
    return cleantext

In [20]:
total_data['clean_tok_text']=total_data['tokenized_text'].apply(lambda x:remove_stopwords(x))
total_data.head()

,full_text,Target,tokenized_text,clean_tok_text
0,Israel starts the new year by demolishing a Pa...,0.0,"[israel, starts, the, new, year, by, demolishi...","[israel, starts, new, year, demolishing, pales..."
1,@realDonaldTrump Trump the White Supremacist...,0.0,"[realdonaldtrump, trump, the, white, supremaci...","[realdonaldtrump, trump, white, supremacist, s..."
2,Please read this. International blackmail from...,1.0,"[please, read, this, international, blackmail,...","[please, read, international, blackmail, ziona..."
3,Netanyahu accidentally calls Israel a 'nuclear...,0.0,"[netanyahu, accidentally, calls, israel, nucle...","[netanyahu, accidentally, calls, israel, nucle..."
4,Jews werent getting their asses kicked on the...,0.0,"[jews, weren, getting, their, asses, kicked, o...","[jews, getting, asses, kicked, streets, nyc, r..."


In [21]:
total_data['clean_text'] = total_data['clean_tok_text'].apply(lambda x:' '.join(x))
total_data

,full_text,Target,tokenized_text,clean_tok_text,clean_text
0,Israel starts the new year by demolishing a Pa...,0.0,"[israel, starts, the, new, year, by, demolishi...","[israel, starts, new, year, demolishing, pales...",israel starts new year demolishing palestinian...
1,@realDonaldTrump Trump the White Supremacist...,0.0,"[realdonaldtrump, trump, the, white, supremaci...","[realdonaldtrump, trump, white, supremacist, s...",realdonaldtrump trump white supremacist st gol...
2,Please read this. International blackmail from...,1.0,"[please, read, this, international, blackmail,...","[please, read, international, blackmail, ziona...",please read international blackmail zionazis a...
3,Netanyahu accidentally calls Israel a 'nuclear...,0.0,"[netanyahu, accidentally, calls, israel, nucle...","[netanyahu, accidentally, calls, israel, nucle...",netanyahu accidentally calls israel nuclear po...
4,Jews werent getting their asses kicked on the...,0.0,"[jews, weren, getting, their, asses, kicked, o...","[jews, getting, asses, kicked, streets, nyc, r...",jews getting asses kicked streets nyc rudygiul...
...,...,...,...,...,...
4015,@DanielHarris he saved more black peoples than...,NaN,"[danielharris, he, saved, more, black, peoples...","[danielharris, saved, black, peoples, daniel, ...",danielharris saved black peoples daniel shut f...
4016,"@PhillyPugilist @DeSeanJackson10 nothing, if y...",NaN,"[phillypugilist, deseanjackson, nothing, if, y...","[phillypugilist, deseanjackson, nothing, kkk, ...",phillypugilist deseanjackson nothing kkk white...
4017,The Nazis murdered millions of Jews because th...,NaN,"[the, nazis, murdered, millions, of, jews, bec...","[nazis, murdered, millions, jews, unwanted, mu...",nazis murdered millions jews unwanted murder m...
4018,"Meanwhile, a group of Orthodox Jews just had t...",NaN,"[meanwhile, group, of, orthodox, jews, just, h...","[meanwhile, group, orthodox, jews, cut, lock, ...",meanwhile group orthodox jews cut lock park city


In [22]:
from transformers import BertTokenizerFast
from transformers import TFBertModel
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [23]:
token_lens = []

for txt in total_data['clean_text'].values:
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
    
max_len=np.max(token_lens)

In [24]:
print(f"MAX TOKENIZED SENTENCE LENGTH: {max_len}")

MAX TOKENIZED SENTENCE LENGTH: 221


In [25]:
token_lens = []

for i,txt in enumerate(total_data['clean_text'].values):
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
    if len(tokens)>80:
        print(f"INDEX: {i}, TEXT: {txt}")  

INDEX: 1898, TEXT: leylakhalid simonvrouwe judgefudge yb oo roni alonma ladyalmagreen joseetje resistdwp phant hebmacman thezionist mistertwyst frontline yes_we_cat themoiety scripteladora david_starof malkaavram boomtown go greatest scot israel danielmazahreh iluvdemocracy jhorn slax baroniesisters sweposten serhumanomiguel swisstechie yankees_rule_ pakeha amoyal priestsanity telavivbroker canpassforsmart dorrit_r rse aron_alee hillshaveeyes searchfeast erezneumark abushlomo breginleif toprantking pohonyangdamai grattan_h charlieluchian rasputinish narrow_p yes arent jew bible jews exist anymore nthere millions fakejews heritage hunters na bible jews nb holocaust nhttps co clducbcump
INDEX: 1908, TEXT: skipaipac mikebloomberg jews jvplive jvpliveny jvpboston jvpbayarea chicagojvp nyujvp jvp_la jvpcolorado jvpgwu jvpindiana jvpmke jvpnola jvpseattle jvpsandiego jvptucson jvptwincities jvpvassar jvpvile ifnotnoworg ifnotnowboston ifnotnow_bayc https co aiqvhyo tx
INDEX: 1935, TEXT: bras

In [26]:
total_data['token_lens'] = token_lens

In [27]:
total_data.shape

(4020, 6)

In [28]:
total_data['Target'].value_counts()

0.0    2473
1.0     743
Name: Target, dtype: int64

In [29]:
total_data.loc[total_data.Target == 1.0, 'DataType'] = "Train"
total_data.loc[total_data.Target == 0.0, 'DataType'] = "Train"
total_data['DataType'] = total_data['DataType'].replace(np.nan, "Test")


In [30]:
train_data = total_data[total_data['DataType']== "Train"]
test_data = total_data[total_data['DataType']== "Test"]
train_data['Target'] = train_data['Target'].astype("int32")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
train_data.Target.unique()

array([0, 1], dtype=int32)

In [32]:
test_data.Target.unique()

array([nan])

In [33]:
X = train_data['clean_text'].values
y = train_data['Target'].values

In [34]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.2, stratify=y_valid, random_state=42)

In [35]:
y_train_le = y_train.copy()
y_valid_le = y_valid.copy()
y_test_le = y_test.copy()

In [36]:
from sklearn import preprocessing

ohe = preprocessing.OneHotEncoder()
y_train = ohe.fit_transform(np.array(y_train).reshape(-1, 1)).toarray()
y_valid = ohe.fit_transform(np.array(y_valid).reshape(-1, 1)).toarray()
y_test = ohe.fit_transform(np.array(y_test).reshape(-1, 1)).toarray()

In [37]:
print(f"TRAINING DATA: {X_train.shape[0]}\nVALIDATION DATA: {X_valid.shape[0]}\nTESTING DATA: {X_test.shape[0]}" )

TRAINING DATA: 2572
VALIDATION DATA: 515
TESTING DATA: 129


In [38]:
MAX_LEN=256

In [39]:
def tokenize(data,max_len=MAX_LEN) :
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer.encode_plus(
            data[i],
            add_special_tokens=True,
            max_length=MAX_LEN,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(np.array(encoded['input_ids']))
        attention_masks.append(np.array(encoded['attention_mask']))
    return np.array(input_ids),np.array(attention_masks)

In [40]:
train_input_ids, train_attention_masks = tokenize(X_train, MAX_LEN)
val_input_ids, val_attention_masks = tokenize(X_valid, MAX_LEN)
test_input_ids, test_attention_masks = tokenize(X_test, MAX_LEN)

In [41]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [42]:
def create_model(bert_model, max_len=MAX_LEN):
    
    ##params###
    opt = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-7)
    loss = tf.keras.losses.CategoricalCrossentropy()
    accuracy = tf.keras.metrics.CategoricalAccuracy()


    input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')
    
    attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')
    
    embeddings = bert_model([input_ids,attention_masks])[1]
    
    output = tf.keras.layers.Dense(2, activation="softmax")(embeddings)
    
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks], outputs = output)
    
    model.compile(opt, loss=loss, metrics=accuracy)
    
    
    return model

In [43]:
model = create_model(bert_model, MAX_LEN)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 256)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_2[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [44]:
train_input_ids = np.stack(train_input_ids,axis =0)
train_attention_masks = np.stack(train_attention_masks, axis = 0)

In [45]:
history_bert = model.fit([train_input_ids,train_attention_masks], y_train, validation_data=([val_input_ids,val_attention_masks], y_valid), epochs=10, batch_size=32)

Epoch 1/10
81/81 [==============================] - 148s 2s/step - loss: 0.4606 - categorical_accuracy: 0.7994 - val_loss: 0.3385 - val_categorical_accuracy: 0.8718
Epoch 2/10
81/81 [==============================] - 133s 2s/step - loss: 0.3189 - categorical_accuracy: 0.8806 - val_loss: 0.2980 - val_categorical_accuracy: 0.8951
Epoch 3/10
81/81 [==============================] - 135s 2s/step - loss: 0.2376 - categorical_accuracy: 0.9129 - val_loss: 0.3743 - val_categorical_accuracy: 0.8485
Epoch 4/10
81/81 [==============================] - 136s 2s/step - loss: 0.1825 - categorical_accuracy: 0.9347 - val_loss: 0.3266 - val_categorical_accuracy: 0.8738
Epoch 5/10
81/81 [==============================] - 136s 2s/step - loss: 0.1376 - categorical_accuracy: 0.9514 - val_loss: 0.3575 - val_categorical_accuracy: 0.8816
Epoch 6/10
81/81 [==============================] - 136s 2s/step - loss: 0.0725 - categorical_accuracy: 0.9751 - val_loss: 0.4114 - val_categorical_accuracy: 0.8854
Epoch 7/10

In [46]:
result_bert = model.predict([test_input_ids,test_attention_masks])

In [47]:
y_pred_bert =  np.zeros_like(result_bert)
y_pred_bert[np.arange(len(y_pred_bert)), result_bert.argmax(1)] = 1

In [48]:
#metrics
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

confusion_matrix(y_test.argmax(1), y_pred_bert.argmax(1))

array([[88, 11],
       [ 8, 22]])

In [49]:
# Generating outputs

In [51]:
result_bert = model.predict([test_input_ids,test_attention_masks])

In [53]:
X_test = test_data['clean_text']

In [57]:
X_test

3216    israel health minister claims coronavirus divi...
3217    greysfaer mean yelling hail victory fuming kik...
3218           https co bnwyz dtkr israel involv one care
3219    dear jews hindus fiercest allies need educate ...
3220    carrieksada baalter updated include hate gener...
                              ...                        
4015    danielharris saved black peoples daniel shut f...
4016    phillypugilist deseanjackson nothing kkk white...
4017    nazis murdered millions jews unwanted murder m...
4018     meanwhile group orthodox jews cut lock park city
4019    ben wheeler e_superdoom_esq member rail union ...
Name: clean_text, Length: 804, dtype: object

In [58]:
test_input_ids, test_attention_masks = tokenize(np.array(X_test), MAX_LEN)

In [59]:
result_bert = model.predict([test_input_ids,test_attention_masks])

In [60]:
y_pred_bert =  np.zeros_like(result_bert)
y_pred_bert[np.arange(len(y_pred_bert)), result_bert.argmax(1)] = 1

In [63]:
test_data['Target'] = y_pred_bert.argmax(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [66]:
test_data['Target'].to_csv(filePath+'predicted.csv')